In [1]:
import os
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Acá se identifica la dirección de los datos de IDEAM en la ciudad Valledupar

base_dir = os.path.join( '.','IDEAMDATA','.','data')
output_dir = os.path.join( '.','output')
ciudad = 'Valledupar'
fuente1 = 'IDEAM'
fuente2 = 'NREL'
serie_csv = os.path.join(base_dir,ciudad,fuente1+'.csv')
serie_pd = pd.read_csv(serie_csv, sep = ';',
                encoding='latin-1')
serie_NREL = os.path.join(base_dir,ciudad,fuente2+'.csv')
data_NREL = pd.read_csv(serie_NREL, sep = ';', encoding='latin-1')

In [3]:
# Función que determina la radiación mensual de la ciudad

def ghi_mensual(serie_horaria):
    anos = sorted(serie_horaria['AÑO'].unique())
    ghi_df = pd.DataFrame(columns=["AÑO", "MES", "DIAS", "GHImes"])
    for ano in anos:
        for mes in range(1,13):
            nombre_energia = list(serie_horaria)[-1]
            dias = sorted(serie_horaria[(serie_horaria['AÑO'] == ano) &
                        (serie_horaria['MES'] == mes)]["DIA"].unique())
            ghi_mes = serie_horaria.loc[((serie_horaria["AÑO"] == ano)
                            & (serie_horaria["MES"] == mes)),
                                        nombre_energia].sum()/1000
            ghi_dict = {"AÑO":ano, "MES":mes, "DIAS":dias[-1],
                        "GHImes":ghi_mes}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
    return ghi_df
# Función que determina la radiación promedio por día de cada més
def ghi_dia(serie_ghi_mensual):
    agnos = sorted(serie_ghi_mensual['AÑO'].unique())
    ghi_df = pd.DataFrame(columns=["AÑO", "MES", "GHIdiario"])
    for agno in agnos:
        for mes in range(1,13):
            dias = serie_ghi_mensual.loc[((serie_ghi_mensual["AÑO"] == agno)
                            & (serie_ghi_mensual["MES"] == mes))
                                         ,"DIAS"].item()
            ghi_diario = serie_ghi_mensual.loc[(
                (serie_ghi_mensual["AÑO"] == agno)
                            & (serie_ghi_mensual["MES"] == mes)),
                "GHImes"].item()/dias
            ghi_dict = {"AÑO":agno, "MES":mes, "GHIdiario":ghi_diario}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
    return ghi_df

In [4]:
ghi_mensual_loc = ghi_mensual(serie_pd)
ghi_diario_loc = ghi_dia(ghi_mensual_loc)
datos_IDEAM = sorted(ghi_diario_loc["GHIdiario"])

In [5]:
 percentil = 0.01
enficc = ghi_diario_loc.quantile(q=percentil, 
                                 numeric_only=True)["GHIdiario"]
print('La ENFICC 99% es: ', enficc)

La ENFICC 99% es:  4.56900737661


# Ahora, se definira la función que ajustará la serie medida insitu contra los datos de la NREL

In [6]:
ghiprueba = ghi_mensual_loc.loc[((ghi_mensual_loc["AÑO"]==2006))]

In [7]:
def ghi_horario (serie):
    nombre_energia = list(serie)[-1]
    ghi_hora = serie.loc[((serie["AÑO"]==2006) & (serie["MES"] <= 2)), nombre_energia]
    return ghi_hora

In [8]:
def ajuste_mcp (serie_IDEAM,serie_NREL):
    datos_IDEAM = ghi_horario(serie_IDEAM)
    datos_NREL = ghi_horario(serie_NREL)
    nombre_energia = list(serie_NREL)[-1]
    mediaX = datos_NREL.mean()
    mediaY = datos_IDEAM.mean()
    sigmaX = datos_IDEAM.var()
    sigmaY = datos_NREL.var()
    IDEAM_conCeros =(mediaY-((sigmaY/sigmaX)*mediaX))+((sigmaY/sigmaX)*(serie_NREL.loc[:,"NREL"]))
    IDEAM_conCeros[IDEAM_conCeros<0]=0
    return IDEAM_conCeros

In [31]:
def ajuste_mcp2 (serie_IDEAM,serie_NREL,ano,mes_fin):
    #Se obtiene el DATAFRAME con el año y mes especificados (Resolución horaria)
    serie_NREL_DATA = pd.DataFrame(columns=['AÑO','MES','DIA','HORA','NREL'])
    serie_IDEAM_DATA = pd.DataFrame(columns=["AÑO", "MES","DIA", "HORA","IDEAM"])
    ##### IDEAM ORGANIZACIÓN
    anos = serie_pd.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'AÑO']
    meses = serie_pd.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'MES']
    dias = serie_pd.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'DIA']
    horas = serie_pd.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'HORA']
    ideams = serie_pd.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'IDEAM']
    ##### NREL ORGANIZACIÓN
    anosNREL = data_NREL.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'AÑO']
    mesesNREL = data_NREL.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'MES']
    diasNREL = data_NREL.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'DIA']
    horasNREL = data_NREL.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'HORA']
    NREL = data_NREL.loc[((serie_pd['AÑO']==ano)&(serie_pd['MES']<=mes_fin)),'NREL']
    
    
    serie_IDEAM_DATA['AÑO'] = anos
    serie_IDEAM_DATA['MES'] = meses
    serie_IDEAM_DATA['DIA'] = anos
    serie_IDEAM_DATA['HORA'] = horas
    serie_IDEAM_DATA['IDEAM'] = ideams
    
    serie_NREL_DATA['AÑO'] = anos
    serie_NREL_DATA['MES'] = meses
    serie_NREL_DATA['DIA'] = anos
    serie_NREL_DATA['HORA'] = horas
    serie_NREL_DATA['NREL'] = NREL
    
    
    serie_IDEAM_DATA.index = pd.RangeIndex(len(serie_IDEAM_DATA))
    serie_NREL_DATA.index = pd.RangeIndex(len(serie_NREL_DATA))
    #Se realiza las operaciones sobre el nuevo DATAFRAME
    mediaX = ideams.mean()
    
    
    return serie_IDEAM_DATA

In [30]:
ajuste_mcp2(serie_pd,data_NREL,2010,1)

,AÑO,MES,DIA,HORA,IDEAM
0,2010,1,2010,0,0.000000
1,2010,1,2010,1,0.000000
2,2010,1,2010,2,0.000000
3,2010,1,2010,3,0.000000
4,2010,1,2010,4,0.000000
5,2010,1,2010,5,0.000000
6,2010,1,2010,6,0.000000
7,2010,1,2010,7,34.099898
8,2010,1,2010,8,226.977448
9,2010,1,2010,9,444.364299


In [ ]:
serie = pd.DataFrame(columns=["AÑO","MES"])
anos = serie_pd.loc[((serie_pd['AÑO']==2010)&(serie_pd['MES']<=2)),'AÑO']
meses = serie_pd.loc[((serie_pd['AÑO']==2010)&(serie_pd['MES']<=2)),'MES']
for ano in anos:
    for mes in meses:
#mes=serie_pd.loc[((serie_pd["AÑO"]==2010)& serie_pd["MES"]==1),["MES"]]
#dia=serie_pd.loc[((serie_pd["AÑO"]==2010)& serie_pd["MES"]==1),["DIA"]]
#hora=serie_pd.loc[((serie_pd["AÑO"]==2010)& serie_pd["MES"]==1),["HORA"]]
#ideam=serie_pd.loc[((serie_pd["AÑO"]==2010)& serie_pd["MES"]==1),["IDEAM"]]
        ghi_dict = {"AÑO":ano,"MES":mes}
        serie = serie.append(ghi_dict, ignore_index= True)

In [ ]:
def plotter1 (serie_IDEAM,serie_NREL):
    IDEAM = np.asarray(ajuste_mcp(serie_IDEAM,serie_NREL))
    NREL = ghi_horario(serie_NREL)
    t = np.asarray(range(len(IDEAM)))
    plt.plot(t,IDEAM)
    plt.plot(t,NREL,'-k')